In [1]:
import pandas as pd
import numpy as np

In [17]:
def str_to_set(x):
    genre_set = set()
    # eval -> python에서 txt data를 내장되어있는 형식으로 바꾸어줌 
    ## 여기서는 리스트 안의 dic 값으로 변환
    for item in eval(x):
        genre_set.add(item['name'])
    return genre_set

# 이렇게 변형 가능
def str_to_set2(x):
    return set(item['name'] for item in eval(x))

def jaccard_similarity(s1, s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = np.sqrt(np.sum(u1_c ** 2) * np.sum(u2_c ** 2))
    if denom != 0:
        return np.sum(u1_c * u2_c)/denom
    else:
        return 0
    
def find_similar_movies (input_title , matrix, n, alpha):
    input_meta = meta.loc[ meta[ 'title'] == input_title].iloc[ 0]
    input_set = input_meta.genres | input_meta.keywords
    
    result = []
    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        this_meta = meta.loc[meta['title'] == this_title].iloc[ 0]
        this_set = this_meta.genres | this_meta.keywords
        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        jaccard = jaccard_similarity(this_set, input_set)
        score = alpha * pearson + ( 1-alpha) * jaccard
        result.append( (this_title, pearson, jaccard, score) )
        
    result.sort(key= lambda r: r[3], reverse= True)
    return result[:n]

In [2]:
meta = pd.read_csv('movies_metadata.csv' )
meta.head()

C:\Users\hyoun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ]
meta.head()

,id,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [4]:
meta = meta.rename(columns={'id':'movieId','original_title': 'title','original_language': 'language'})
meta.head()

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [5]:
meta = meta.loc[meta['language'] == 'en',:]
meta.head()

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [6]:
meta.movieId = pd.to_numeric(meta.movieId)
meta.movieId

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45459    222848
45460     30840
45463     67758
45464    227506
45465    461257
Name: movieId, Length: 32269, dtype: int64

In [8]:
meta.genres = meta.genres.apply(str_to_set)
meta

,movieId,title,language,genres
0,862,Toy Story,en,"{Animation, Family, Comedy}"
1,8844,Jumanji,en,"{Family, Fantasy, Adventure}"
2,15602,Grumpier Old Men,en,"{Comedy, Romance}"
3,31357,Waiting to Exhale,en,"{Comedy, Romance, Drama}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Drama, Action, Romance}"
45463,67758,Betrayal,en,"{Drama, Thriller, Action}"
45464,227506,Satana likuyushchiy,en,{}


In [12]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [13]:
keywords.keywords = keywords.keywords.apply(str_to_set)
keywords.head()

,id,keywords
0,862,"{new toy, friendship, friends, toy, boy, rival..."
1,8844,"{based on children's book, recluse, board game..."
2,15602,"{fishing, old men, best friend, duringcreditss..."
3,31357,"{interracial relationship, based on novel, div..."
4,11862,"{aging, baby, midlife crisis, mother daughter ..."


In [14]:
keywords = keywords.rename(columns={'id': 'movieId'})
keywords.movieId = pd.to_numeric(keywords.movieId)
keywords.head()

,movieId,keywords
0,862,"{new toy, friendship, friends, toy, boy, rival..."
1,8844,"{based on children's book, recluse, board game..."
2,15602,"{fishing, old men, best friend, duringcreditss..."
3,31357,"{interracial relationship, based on novel, div..."
4,11862,"{aging, baby, midlife crisis, mother daughter ..."


In [15]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta.head()

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Animation, Family, Comedy}","{new toy, friendship, friends, toy, boy, rival..."
1,8844,Jumanji,en,"{Family, Fantasy, Adventure}","{based on children's book, recluse, board game..."
2,15602,Grumpier Old Men,en,"{Comedy, Romance}","{fishing, old men, best friend, duringcreditss..."
3,31357,Waiting to Exhale,en,"{Comedy, Romance, Drama}","{interracial relationship, based on novel, div..."
4,11862,Father of the Bride Part II,en,{Comedy},"{aging, baby, midlife crisis, mother daughter ..."


In [16]:
dk = meta.loc[meta.title == 'The Dark Knight'].iloc[0]
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0]
pd.concat([dk, dkr], axis=1).T

,movieId,title,language,genres,keywords
10278,155,The Dark Knight,en,"{Crime, Drama, Thriller, Action}","{criminal mastermind, based on comic, chaos, s..."
14315,49026,The Dark Knight Rises,en,"{Crime, Drama, Thriller, Action}","{villainess, terrorist, hostage drama, catwoma..."


In [18]:
jaccard_similarity(dk.genres|dk.keywords,
dkr.genres|dkr.keywords)

0.37142857142857144

In [24]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [25]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [26]:
ratings.movieId = pd.to_numeric(ratings.movieId)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [27]:
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III


In [28]:
matrix = ratings.pivot_table(index= 'userId', columns= 'title',values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']
pearson_similarity(dk_rating, pk_rating)

0.773565934694095

In [30]:
result = find_similar_movies('The Dark Knight', matrix, 10, 0.3)
pd.DataFrame(result, columns = ['title', 'pearson', 'jaccard', 'score'])

,title,pearson,jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399
